In [19]:
# Import dependencies
from sleeperpy import User, Leagues
import pandas as pd

In [20]:
# User inputs username and selects a league from landing page
username = "dirtywizard"
# Get User's Sleeper ID and Sleeper leagues
user_id = User.get_user(username)['user_id']
leagues = Leagues.get_all_leagues(user_id, 'nfl', 2024)
league_name_list = [x['name'] for x in leagues]
# Show the user the list of names and have them select one
print(league_name_list)
league_name = "No Coffee Sharps D.F.F.L."
# Find the league id for the selected league
league_id = [x['league_id'] for x in leagues if x['name'] == league_name][0]

['No Coffee Sharps D.F.F.L.']


In [21]:
# Receive user ID & league ID in GET request, then fetch league data from Sleeper API
selected_league = Leagues.get_league(league_id)
scoring_settings = selected_league['scoring_settings']
# Get the rosters for the league
rosters = Leagues.get_rosters(league_id)
users = Leagues.get_users(league_id)
print(f'league_id: {league_id}')
print(f'user_id: {user_id}')

league_id: 1049754172834455552
user_id: 725907041763303424


In [22]:
# Bring in the franchise_id for each roster
roster_df = pd.DataFrame(rosters)
roster_df = roster_df[['players', 'owner_id']]
# Explode the roster_df to get a row for each player on each roster
roster_df = roster_df.explode('players')
roster_df['players'] = roster_df['players'].astype(str)
roster_df = roster_df.rename(columns={'players': 'id_sleeper'})
roster_df.head()

KeyError: 'is_user'

In [24]:
# Get info about each franchise
franchises = pd.DataFrame()
franchises['owner_id'] = [x['user_id'] for x in users]
franchises['owner_name'] = [x['display_name'] for x in users]
franchises['team_name'] = [x['metadata']['team_name'] if 'team_name' in x['metadata'] else x['display_name'] for x in users]
# Create a True/False column for whether the owner is the user
franchises['is_user'] = False
franchises.loc[franchises['owner_id'] == user_id, 'is_user'] = True
franchises.duplicated()

,id_sleeper,owner_id
0,11575,208058775418961920
0,11643,208058775418961920
0,8139,208058775418961920
0,10871,208058775418961920
0,10235,208058775418961920


In [7]:
# Merge roster_df with franchises to get the owner_name and team_name for each player
roster_df = pd.merge(roster_df, franchises, on='owner_id')
roster_df.head()

,owner_id,owner_name,team_name,is_user
0,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
1,342869377223385088,nrector,The Fort Worth Fremen,False
2,412722740429082624,jccarm,DangeRUSS Last Ride,False
3,452577139540094976,2014champ,2014champ,False
4,461323803130064896,cdulgar,Idk much about soccer,False
5,725907041763303424,dirtywizard,Croccity Body Snatchers,True
6,904381167934095360,mdwglobe,GusTheBus,False
7,904398822170636288,SpacemnSpiff,Saskatoon Squatches,False
8,904399374891855872,comebackking15,Comeback 👑’s,False
9,904400906127458304,TrulyUhle,Verdanks Vacqueros FFC,False


In [8]:
# Read RDS database 
# Select id_sleeper, full_name, age, team, number, position from players where id_sleeper in rostered_players
allplayers = pd.read_csv('test_data/rds_players.csv', dtype={'id_sleeper': 'str'}) 

# Read RDS database
predictions = pd.read_csv('test_data/rds_predictions.csv', dtype={'id_sleeper': 'str'})
# Get a list of rostered players 
rostered_players = roster_df['id_sleeper'].tolist()
# Get a list of the user's players
user_players = roster_df.loc[roster_df['is_user'] == True, 'id_sleeper'].tolist()
# Filter the predictions to only include players on the user's roster or on waivers
predictions = predictions.loc[(~predictions['id_sleeper'].isin(rostered_players)) | (predictions['id_sleeper'].isin(user_players))]
predictions = predictions.merge(allplayers[['id_sleeper', 'full_name', 'age', 'team', 'number', 'position']], on='id_sleeper', how='left')

# Multiply each prediction column by the corresponding scoring setting
for column in predictions.columns:
    if column in scoring_settings:
        predictions[column] = predictions[column] * scoring_settings[column]
# Sum up the total points for each column in the scoring settings
predictions['predicted_fantasy_points'] = predictions[list(scoring_settings.keys())].sum(axis=1)
predictions = predictions.drop(columns=list(scoring_settings.keys())).drop(columns=['index_predictions'])
predictions.sort_values(by='predicted_fantasy_points', ascending=False, inplace=True, ignore_index=True)
predictions = predictions[['id_sleeper', 'full_name', 'age', 'team', 'number', 'position', 'week_of_season', 'predicted_fantasy_points']]
predictions.head()

,id_sleeper,owner_id,owner_name,team_name,is_user
0,11575,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
1,11643,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
2,8139,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
3,10871,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
4,10235,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False


In [9]:
# Merge the predictions with the roster_df to get the owner_name and team_name for each player
complete = pd.merge(predictions, roster_df, on='id_sleeper', how='left')
# Mark the unrostered players as 'Free Agents'
complete.loc[complete['owner_id'].isnull(), 'team_name'] = 'Free Agents'
complete.loc[complete['owner_id'].isnull(), 'owner_name'] = 'Free Agents'
complete.head()

,index_predictions,id_sleeper,week_of_season,pass_cmp,pass_yd,pass_td,pass_int,rush_att,rush_yd,rush_td,...,def_2pt,st_fum_rec,st_ff,st_td,fum,fum_rec_td,owner_id,owner_name,team_name,is_user
0,5859_0,5859,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1097007663277379584,Jeffrito17,Jeffrito17,False
1,5859_1,5859,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1097007663277379584,Jeffrito17,Jeffrito17,False
2,5859_2,5859,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1097007663277379584,Jeffrito17,Jeffrito17,False
3,5859_3,5859,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1097007663277379584,Jeffrito17,Jeffrito17,False
4,5859_5,5859,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1097007663277379584,Jeffrito17,Jeffrito17,False


In [10]:
# Convert the dataframe to json
waiverwire_result = complete.to_dict(orient='records')
# Write to file
complete.to_json('test_data/waiverwire.json', orient='records')
waiverwire_result

[{'index_predictions': '5859_0',
  'id_sleeper': '5859',
  'week_of_season': 0,
  'pass_cmp': 0.0,
  'pass_yd': 0.0,
  'pass_td': 0.0,
  'pass_int': 0.0,
  'rush_att': 0.0,
  'rush_yd': 0.0,
  'rush_td': 0.0,
  'fum_lost': 0.2,
  'rec': 5.8,
  'rec_yd': 82.0,
  'rec_td': 0.5,
  'xpm': 0.0,
  'fgm': 0.0,
  'fgm_0_19': 0.0,
  'fgm_20_29': 0.0,
  'fgm_30_39': 0.0,
  'fgm_40_49': 0.0,
  'fgm_50p': 0.0,
  'fgmiss': 0.0,
  'sack': 0.0,
  'def_st_td': 0.0,
  'safe': 0.0,
  'pr_yd': 0.0,
  'kr_yd': 0.0,
  'fgm_yds_over_30': 0.0,
  'pts_allow_0': 0.0,
  'pts_allow_1_6': 0.0,
  'pts_allow_7_13': 0.0,
  'pts_allow_14_20': 0.0,
  'pts_allow_21_27': 0.0,
  'pts_allow_28_34': 0.0,
  'pts_allow_35p': 0.0,
  'yds_allow_0_100': 0.0,
  'yds_allow_100_199': 0.0,
  'yds_allow_200_299': 0.0,
  'yds_allow_400_449': 0.0,
  'yds_allow_450_499': 0.0,
  'yds_allow_500_549': 0.0,
  'yds_allow_550p': 0.0,
  'bonus_rush_yd_100': 0.0,
  'bonus_rush_yd_200': 0.0,
  'bonus_rec_yd_100': 0.0,
  'bonus_rec_yd_200': 0.0,

In [15]:
# Troubleshoot
# Receive user ID & league ID in GET request, then fetch league data from Sleeper API
selected_league = Leagues.get_league(league_id)
roster_positions = selected_league['roster_positions']
scoring_settings = selected_league['scoring_settings']
# Get the rosters for the league
rosters = Leagues.get_rosters(league_id)
users = Leagues.get_users(league_id)

# Bring in the franchise_id for each roster
roster_df = pd.DataFrame(rosters)
roster_df = roster_df[['players', 'owner_id']]
# Explode the roster_df to get a row for each player on each roster
roster_df = roster_df.explode('players')
roster_df['players'] = roster_df['players'].astype(str)
roster_df = roster_df.rename(columns={'players': 'id_sleeper'})

# Get info about each franchise
franchises = pd.DataFrame()
franchises['owner_id'] = [x['user_id'] for x in users]
franchises['owner_name'] = [x['display_name'] for x in users]
franchises['team_name'] = [x['metadata']['team_name'] if 'team_name' in x['metadata'] else x['display_name'] for x in users]
# Create a True/False column for whether the owner is the user
franchises['is_user'] = False
franchises.loc[franchises['owner_id'] == user_id, 'is_user'] = True

# Merge roster_df with franchises to get the owner_name and team_name for each player
roster_df = pd.merge(roster_df, franchises, on='owner_id')
print(len(roster_df))
roster_df.drop_duplicates(subset=['id_sleeper'], inplace=True)
print(len(roster_df))
roster_df

419
419


,id_sleeper,owner_id,owner_name,team_name,is_user
0,11575,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
1,11643,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
2,8139,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
3,10871,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
4,10235,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
...,...,...,...,...,...
414,8155,904399374891855872,comebackking15,Comeback 👑’s,False
415,8219,904399374891855872,comebackking15,Comeback 👑’s,False
416,9225,904399374891855872,comebackking15,Comeback 👑’s,False
417,LAR,904399374891855872,comebackking15,Comeback 👑’s,False
